# Importing packages

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import tensorflow as tf

from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, mean_absolute_error, r2_score,plot_confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, train_test_split, KFold,GridSearchCV,RepeatedStratifiedKFold 
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, RobustScaler, OrdinalEncoder
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.feature_selection._base import SelectorMixin
from sklearn.feature_extraction.text import _VectorizerMixin

In [2]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
import warnings # Ignore alert messages
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
plt.rcParams.update({
    'font.size': 18,
    'axes.linewidth':2,
    'axes.titlesize': 20,
    'lines.linewidth' : 1.5,
    'lines.markersize' : 10,
    'figure.figsize': (25,10),
    'xtick.labelsize': 15, 
    'ytick.labelsize': 15,
    'font.family': 'Arial',
    'legend.fontsize':14,
    'axes.labelsize':20,
    'axes.grid':True,
})

# System description

An experimental rig to demonstrate filter clogging failure has been constructed and consists of the following major components: 

- Pump,
- Liquid tanks,
- Tank stirrer,
- Pulsation dampener,
- Filter,
- Pressure and flow rate sensors,
- Data acquisition system connected to a computer.

The experiment rig is a circuit composed of a pump flowing a liquid from a tank to another through a filter. The circuit is instrumented with sensors able to monitor the following variables:

- Flow rate,
- Liquid pressure before the filter 
- Liquid pressure after the filter. 
  
The fluid injected in the system is a suspension composed by Polyetheretherketone (PEEK) particles and water with different concentrations. To eliminate possible pulsations in the flow the circuit includes a dampener. The figures below depict the employed experimental rig as well as the filter.

![title](Figures/Experimental_rig.png) ![title](Figures/Filter.png)

The main components applied to construct this experimental rig are the following:

- **Pump:** Since the system will involve contaminants in the fluid, a peristaltic pump has been used as its mechanism is more tolerant to particles in the liquid. A Masterflex® SN-77921-70 (Drive: 07523-80, Two Heads: 77200-62, Tubing: L/S© 24) model peristaltic pump was installed in the system to maintain the flow of the prepared suspension. The pump is a positive displacement source, providing a flow rate ranging from 0.28 to 1700 ml/min (i.e. from 0.1 to 600 RPM).
- 
- **Dampener:** The aim of using rigid tubing is to prevent the system from unwanted tube expansion due to pressure build up, which affects the actual pressure build up generated from filter clogging. A Masterflex® pulse dampener is installed on the downstream side of pump to eliminate any pulsation in flow.A majority of the system is furnished with a rigid polypropylene tubing, whereas the pump side is covered with a flexible Tygon® LFL pump tubing.

- **Particles:** The suspension is composed of Polyetheretherketone (PEEK) particles and water. PEEK particles have a density (1.3g/cm3) close to that of room temperature water and have significantly low water absorption level (0.1% / 24 hours, ASTM D570). Having a low water absorption level will prevent particles expanding when they mix with water. Subsequently, closer density with water allows particles to suspend longer in water.

- **Flow Rate Sensor:** A GMAG100 series electromagnetic flow meter (measurement range: 3 – 25,000 millilitres per minute) is installed in the system to keep track of the flow rate in the system.
  
- **Pressure Sensors:** Upstream and downstream Ashcroft® G2 pressure transducers (measurement range: 0 – 100 PSI) are installed in the system to capture the pressure drop (i.e. ‘ΔP’) across the filter, which is considered as the main indicator of clogging.
  
- **Filter:** the filter has a pore mesh size of 125µm.

# Objective

1)  Predict when the filter is fully clogged;
2)  Determine the remaining useful life (RUL) of the filter;
3)  Assess model performance for decreasing dataset sizes (100%,75%,50%,25% of all experiments to create the model);
4)  Assess the reliability of the RUL metric with a confidence interval (extra)

# Importing datasets